In [4]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    if int(article_id) == 12:
        for word in words:
            print "%s\t%d" % (word.lower(), 1)

Writing mapper1.py


In [5]:
%%writefile reducer1.py

import sys

current_key = None
word_sum = 0
N=0
Nt=0
tf=0

with open("stop_words_en.txt") as stopfile:
    stopwords = stopfile.readlines()
stopwords = list(map(lambda i:i.strip(),stopwords))
    
for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            if not(current_key in stopwords):
                #print "%s\t%d" % (current_key, word_sum)
                if current_key=="labor":
                    Nt=word_sum
                N=N+word_sum
            #print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    #print "%s\t%d" % (current_key, word_sum)
    if current_key=="labor":
        Nt=word_sum
    N=N+word_sum
    tf=float(Nt)/float(N)
    print tf 
    #print Nt
    #print "tf=%d" %tf

Writing reducer1.py


In [6]:
%%writefile combiner1.py

import sys

current_key = None
word_sum = 0
    
for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing combiner1.py


In [50]:
%%writefile mapper2.py

import sys
import re
import math

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

Dt=0.0
idf=0.0
N=0.0

for line in sys.stdin:
    flag=0
    N=N+1
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        #print "%s\t%d" % (word.lower(), 1)
        if word.lower()=="labor" and flag==0:
            Dt=Dt+1
            flag=1
idf=math.log(N/Dt)
print idf

Overwriting mapper2.py


In [51]:
%%bash

OUT_DIR="tf_result_"$(date +"%s%6N")
NUM_REDUCERS=1
OUT_DIR2="dt_result_"$(date +"%s%6N")
NUM_REDUCERS2=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Term Count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py,combiner1.py,/datasets/stop_words_en.txt \
    -mapper "python mapper1.py" \
    -combiner "python combiner1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

tf=$(hdfs dfs -cat ${OUT_DIR}/part-00000)

hdfs dfs -rm -r -skipTrash ${OUT_DIR2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Doc Frequency" \
    -files mapper2.py \
    -mapper "python mapper2.py" \
    -numReduceTasks 0 \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR2} > /dev/null

idf=$(hdfs dfs -cat ${OUT_DIR2}/part-00000)

tfidf=$(echo "${tf}*${idf}" |bc)

printf '%.10f\n' ${tfidf}

0.0053071615


rm: `tf_result_1515693724648636': No such file or directory
18/01/11 18:02:08 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/11 18:02:08 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/11 18:02:08 INFO mapred.FileInputFormat: Total input files to process : 1
18/01/11 18:02:08 INFO mapreduce.JobSubmitter: number of splits:2
18/01/11 18:02:08 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1515627025943_0026
18/01/11 18:02:09 INFO impl.YarnClientImpl: Submitted application application_1515627025943_0026
18/01/11 18:02:09 INFO mapreduce.Job: The url to track the job: http://9de2cfc801c0:8088/proxy/application_1515627025943_0026/
18/01/11 18:02:09 INFO mapreduce.Job: Running job: job_1515627025943_0026
18/01/11 18:02:15 INFO mapreduce.Job: Job job_1515627025943_0026 running in uber mode : false
18/01/11 18:02:15 INFO mapreduce.Job:  map 0% reduce 0%
18/01/11 18:02:24 INFO mapreduce.Job:  map 100% reduce 0%
18/01/11 18:02:29 I